## Extração, Transformação e Carga (ETL)

### ATENÇÃO!!
Esta ETL exige **21 GB de espaço** em disco e **demora mais de 8 horas** se o seu equipamento for similar ao do nosso experimento: (Processador i3 Dual Core 2,1GHz e memória RAM DDR3 de 8 GB e 1.333MHz)

Esta etapa somente é necessária caso queira conhecer todo o processo de tratamento do conjunto de dados.

Você pode pular esta etapa e ir direto para o notebook [Análise da Carga de Trabalho da Máquina](machine_workload_analysis.ipynb) que utiliza o arquivo 'loaded_m_3330.csv' gerado após o tratamento.

In [ ]:
# Baixe o conjunto de dados (1,7 GB)
!wget http://clusterdata2018pubcn.oss-cn-beijing.aliyuncs.com/machine_usage.tar.gz
    
# Descompacte o arquivo (9 GB)
!tar -zxvf machine_usage.tar.gz

# Execute o script para separar as maquinas (10 GB)
!python split_machines.py

In [31]:
# Quantidade de máquinas extraídas
import os

files = os.listdir(r'machines')
len(files)

4023

In [2]:
# Identifica máquinas ausentes no conjunto de dados
import os

files = os.listdir(r'machines')
missing = [n+1 for n in range(4034) if f'm_{n+1}.csv' not in files]
missing

[192, 374, 1040, 1074, 1483, 1495, 1807, 2715, 3344, 3404, 3520]

In [3]:
# Lista os arquivos de cada máquina ordenando pela quantidade de registros
import pandas as pd
missing = [192, 374, 1040, 1074, 1483, 1495, 1807, 2715, 3344, 3404, 3520]
machines = [f'm_{n+1}.csv' for n in range(4034) if n+1 not in missing]
records = [len(pd.read_csv(r'machines/' + machine)) for machine in machines]
machine_records = (
    pd.DataFrame({'machine':machines, 'records':records})
    .sort_values('records')
    .reset_index(drop=True)
)
             
print(machine_records)

         machine  records
0     m_3539.csv     3569
1     m_3306.csv     6948
2     m_2540.csv     9826
3      m_556.csv    18318
4      m_669.csv    22537
...          ...      ...
4018   m_103.csv    64675
4019  m_2773.csv    64698
4020  m_2967.csv    64787
4021   m_694.csv    65041
4022  m_3330.csv    65043

[4023 rows x 2 columns]


In [4]:
# Escolhemos a máquina 'm_3330.csv' pois é a que contêm mais registros
import matplotlib.pyplot as plt
machine = 'm_3330.csv'
df = pd.read_csv(r'machines/' + machine, 
                 names = ['machine_id', 'time_stamp', 'cpu', 'mem', 'mem_gps', 
                          'mkpi', 'net_in', 'net_out', 'disk_io_percent'])
df

,machine_id,time_stamp,cpu,mem,mem_gps,mkpi,net_in,net_out,disk_io_percent
0,m_3330,10,28,91,NaN,NaN,45.21,38.10,3
1,m_3330,130,20,91,NaN,NaN,45.21,38.10,4
2,m_3330,160,19,92,NaN,NaN,45.22,38.10,4
3,m_3330,170,16,92,NaN,NaN,45.22,38.10,4
4,m_3330,280,20,92,NaN,NaN,45.22,38.10,4
...,...,...,...,...,...,...,...,...,...
65039,m_3330,690860,30,92,7.08,0.0,62.64,55.28,7
65040,m_3330,690870,34,92,3.65,0.0,62.64,55.28,7
65041,m_3330,690880,32,92,4.11,0.0,62.64,55.28,7
65042,m_3330,690910,30,91,5.19,0.0,62.64,55.28,4


In [24]:
# Transforma o conjunto de dados do arquivo 'machines/m_3330.csv' 
# e faz a carga do resultado no arquivo 'loaded_m_3330.csv'.
import pandas as pd

def dataframe(file):
    df = pd.read_csv(r'machines/' + file, 
                     usecols=[1, 2, 3],
                     names=['time_stamp', 'cpu', 'mem'])
    df_len = len(df)
    df.time_stamp = pd.to_datetime(df.time_stamp, unit='s')
    df.set_index('time_stamp', inplace=True)
    df = df.resample('5min').interpolate()
    df = df.dropna()
    if len(files) == 4023:
        df.to_csv('loaded_'+file)
    return df
dataframe(file)

,cpu,mem
time_stamp,,
1970-01-01 00:00:00,17.0,92.0
1970-01-01 00:05:00,20.0,92.0
1970-01-01 00:10:00,34.0,95.0
1970-01-01 00:15:00,27.0,93.0
1970-01-01 00:20:00,33.0,95.0
...,...,...
1970-01-08 23:35:00,21.0,89.0
1970-01-08 23:40:00,25.0,90.0
1970-01-08 23:45:00,19.0,88.0
